In [150]:
from qiskit import Aer, QuantumCircuit, QuantumRegister, transpile, execute, IBMQ
import numpy as np
from qiskit.providers.aer.backends.aerbackend import AerBackend
from qiskit.quantum_info import Operator
from qiskit.visualization import plot_histogram
from scipy import linalg

In [151]:
simulator = Aer.get_backend('qasm_simulator')

In [152]:
# Full Adder
# Register: 0 - x, 1 - y, 2 - carry, 3 - out
def simple_full_adder() -> QuantumCircuit:
     full_adder = QuantumCircuit(4, name='full_adder')
     full_adder.ccx(0, 1, 3)
     full_adder.cx(0, 1)
     full_adder.ccx(1, 2, 3)
     full_adder.cx(1, 2)
     full_adder.cx(0, 1)
     full_adder.swap(2, 3)
     return full_adder

full_adder_gate = simple_full_adder().to_gate()
full_adder_instruction = simple_full_adder().to_instruction()

In [153]:
def full_adder_circuit(length: int) -> QuantumCircuit:
    if length < 1:
        raise Exception('Register length must be positive number')

    if length == 1:
        return simple_full_adder()

    x = QuantumRegister(length, 'x')
    y = QuantumRegister(length, 'y')
    out = QuantumRegister(length, 'out')
    carry = QuantumRegister(1, 'carry')

    adder = QuantumCircuit(x, y, out, carry, name=format("add_%s", str(length)))

    for i in range(0, length):
        adder.append(full_adder_instruction, [x[i], y[i], out[i], carry[0]])

    return adder

In [154]:
def xor_circuit(length: int) -> QuantumCircuit:
    if length < 1:
        raise Exception('Register length must be positive number')

    x = QuantumRegister(length, 'x')
    y = QuantumRegister(length, 'y')
    out = QuantumRegister(length, 'out')

    xor = QuantumCircuit(x, y, out, name=format("xor_%s", str(length)))

    for i in range(0, length):
        xor.cx(x[i], out[i])
        xor.cx(y[i], out[i])

    return xor

In [155]:
def vector_to_indices(v: int) -> list:
    ind = []
    for i in range(8):
        if (v >> i) & 0x1 == 0x1:
            ind.append(i)
    return ind


def s_box_circuit(anf: list, box_name: str) -> QuantumCircuit:
    assert len(anf) == (1 << 8)

    x = QuantumRegister(8, 'x')
    out = QuantumRegister(8, 'out')

    circuit = QuantumCircuit(x, out, name="s_box_" + box_name)

    for i in range(1 << 8):
        if anf[i] != 0:
            if i == 0:
                for ind in vector_to_indices(anf[i]):
                    circuit.x(out[ind])
            else:
                for ind in vector_to_indices(anf[i]):
                    circuit.mcx(x[vector_to_indices(i)], out[ind])

    return circuit


In [156]:
def add_f8_circuit() -> QuantumCircuit:
    return xor_circuit(8)

In [207]:
# Reduction polynomial x^8 + x^4 + x^3 + x^2 + 1

# Reduction matrix

Q = np.array([
    [1, 0, 1, 1, 1, 0, 0, 0],
    [0, 1, 0, 1, 1, 1, 0, 0],
    [0, 0, 1, 0, 1, 1, 1, 0],
    [0, 0, 0, 1, 0, 1, 1, 1],
    [1, 0, 1, 1, 0, 0, 1, 1],
    [1, 1, 1, 0, 0, 0, 0, 1],
    [1, 1, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0]
])

In [208]:
print(Q.transpose())


[[1 0 0 0 1 1 1 0]
 [0 1 0 0 0 1 1 0]
 [1 0 1 0 1 1 0 0]
 [1 1 0 1 1 0 0 0]
 [1 1 1 0 0 0 1 0]
 [0 1 1 1 0 0 0 0]
 [0 0 1 1 1 0 0 0]
 [0 0 0 1 1 1 0 0]]


In [209]:
Qt = Q.copy().transpose()

In [219]:
print(Qt)

[[1 0 0 0 1 1 1 0]
 [0 1 0 0 0 1 1 0]
 [0 0 1 0 0 0 1 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0]]


In [211]:
Qt[2] = Qt[2] ^ Qt[0] # 0 -> 2
Qt[3] = Qt[3] ^ Qt[0] # 0 -> 3
Qt[4] = Qt[4] ^ Qt[0] # 0 -> 4

In [212]:
Qt[3] = Qt[3] ^ Qt[1]
Qt[4] = Qt[4] ^ Qt[1]
Qt[5] = Qt[5] ^ Qt[1]

In [213]:
Qt[4] = Qt[4] ^ Qt[2]
Qt[5] = Qt[5] ^ Qt[2]
Qt[6] = Qt[6] ^ Qt[2]

In [215]:
Qt[5] = Qt[5] ^ Qt[3]
Qt[6] = Qt[6] ^ Qt[3]
Qt[7] = Qt[7] ^ Qt[3]

In [216]:
Qt[6] = Qt[6] ^ Qt[4]
Qt[7] = Qt[7] ^ Qt[4]

In [218]:
Qt[7] = Qt[7] ^ Qt[5]

In [220]:
print(Qt)

[[1 0 0 0 1 1 1 0]
 [0 1 0 0 0 1 1 0]
 [0 0 1 0 0 0 1 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0]]


In [239]:
Qtt = np.array(Qt).transpose()

In [245]:
print(Qtt)

[[1 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0]]


In [241]:
Qtt[4] = Qtt[4] ^ Qtt[0]
Qtt[5] = Qtt[5] ^ Qtt[0]
Qtt[6] = Qtt[6] ^ Qtt[0]

In [242]:
Qtt[5] = Qtt[5] ^ Qtt[1]
Qtt[6] = Qtt[6] ^ Qtt[1]

In [244]:
Qtt[6] = Qtt[6] ^ Qtt[2]


In [246]:
print(Qtt)

[[1 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0]]


In [247]:
def qt_reduction_circuit() -> QuantumCircuit:
    qt = QuantumCircuit(8, name="qt_reduction")

    qt.cx(4, 0)
    qt.cx(5, 0)
    qt.cx(6, 0)
    qt.cx(5, 1)
    qt.cx(6, 1)
    qt.cx(6, 2)

    qt.barrier()

    qt.cx(5, 7)
    qt.cx(4, 7)
    qt.cx(4, 6)
    qt.cx(3, 7)
    qt.cx(3, 6)
    qt.cx(3, 5)
    qt.cx(2, 6)
    qt.cx(2, 5)
    qt.cx(2, 4)
    qt.cx(1, 5)
    qt.cx(1, 4)
    qt.cx(1, 3)
    qt.cx(0, 4)
    qt.cx(0, 3)
    qt.cx(0, 2)

    return qt

qt_reduction_circuit().draw()

┌───┐┌───┐┌───┐                ░                                         »
q_0: ┤ X ├┤ X ├┤ X ├────────────────░─────────────────────────────────────────»
     └─┬─┘└─┬─┘└─┬─┘┌───┐┌───┐      ░                                         »
q_1: ──┼────┼────┼──┤ X ├┤ X ├──────░─────────────────────────────────────────»
       │    │    │  └─┬─┘└─┬─┘┌───┐ ░                                         »
q_2: ──┼────┼────┼────┼────┼──┤ X ├─░─────────────────────────────────■────■──»
       │    │    │    │    │  └─┬─┘ ░                                 │    │  »
q_3: ──┼────┼────┼────┼────┼────┼───░──────────────────■────■────■────┼────┼──»
       │    │    │    │    │    │   ░                  │    │    │    │    │  »
q_4: ──■────┼────┼────┼────┼────┼───░────────■────■────┼────┼────┼────┼────┼──»
            │    │    │    │    │   ░        │    │    │    │  ┌─┴─┐  │  ┌─┴─┐»
q_5: ───────■────┼────■────┼────┼───░───■────┼────┼────┼────┼──┤ X ├──┼──┤ X ├»
                 │         │    │   ░   │    │  ┌─┴─┐  │  ┌─┴─┐└───┘┌─┴─┐└───┘»
q_6: ────────────■─────────■────■───░───┼────┼──┤ X ├──┼──┤ X ├─────┤ X ├─────»
                                    ░ ┌─┴─┐┌─┴─┐└───┘┌─┴─┐└───┘     └───┘     »
q_7: ───────────────────────────────░─┤ X ├┤ X ├─────┤ X ├────────────────────»
                                    ░ └───┘└───┘     └───┘                    »
«                                        
«q_0: ──────────────────────■────■────■──
«                           │    │    │  
«q_1: ───────■────■────■────┼────┼────┼──
«            │    │    │    │    │  ┌─┴─┐
«q_2: ──■────┼────┼────┼────┼────┼──┤ X ├
«       │    │    │  ┌─┴─┐  │  ┌─┴─┐└───┘
«q_3: ──┼────┼────┼──┤ X ├──┼──┤ X ├─────
«     ┌─┴─┐  │  ┌─┴─┐└───┘┌─┴─┐└───┘     
«q_4: ┤ X ├──┼──┤ X ├─────┤ X ├──────────
«     └───┘┌─┴─┐└───┘     └───┘          
«q_5: ─────┤ X ├─────────────────────────
«          └───┘                         
«q_6: ───────────────────────────────────
«                                        
«q_7: ───────────────────────────────────
«